In [1]:
!pip install datasets transformers rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 31.7 MB/s 
     |████████████████████████████████| 4.4 MB 48.4 MB/s 
     |████████████████████████████████| 1.1 MB 54.1 MB/s 
     |████████████████████████████████| 140 kB 73.3 MB/s 
     |████████████████████████████████| 101 kB 15.1 MB/s 
     |████████████████████████████████| 212 kB 57.8 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
     |████████████████████████████████| 127 kB 38.9 MB/s 
     |████████████████████████████████| 6.6 MB 30.9 MB/s 
     |████████████████████████████████| 271 kB 12.5 MB/s 
     |████████████████████████████████| 144 kB 76.9 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existin

In [3]:
!cp kaggle.json ~/.kaggle/kaggle.json
!kaggle datasets download -d fabiochiusano/medium-articles

 96% 353M/369M [00:15<00:00, 27.8MB/s]
100% 369M/369M [00:15<00:00, 24.8MB/s]


## Load the dataset

In [4]:
import transformers
import numpy as np
from datasets import load_dataset, load_metric

In [7]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [9]:
medium_datasets_cleaned = medium_datasets.filter(lambda example: (len(example['text']) >= 500) and (len(example['title']) >= 20))
tokenized_datasets = medium_datasets_cleaned.map(preprocess_data, batched=True)

Parameter 'function'=<function <lambda> at 0x7f0017840a70> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [11]:
max_input_length = 512
max_target_length = 64
metric = load_metric("rouge")

In [37]:
import pandas as pd

In [38]:
data=pd.read_csv('/content/CNN_Articles.csv')

In [54]:
data.shape

(11490, 3)

In [40]:
CNN_dataset = load_dataset("csv", data_files="/content/CNN_Articles.csv")

Using custom data configuration default-62f7e103dd3f3710


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-62f7e103dd3f3710/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

**Dataset Train and validation split**

In [41]:
datasets_train_test = CNN_dataset["train"].train_test_split(test_size=1000)
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=1000)
CNN_dataset["train"] = datasets_train_validation["train"].shuffle().select(range(9000))
CNN_dataset["validation"] = datasets_train_validation["test"].shuffle().select(range(1000))
CNN_dataset["test"] = datasets_train_test["test"].shuffle().select(range(1000))

**Data Preprocessing**

In [42]:
prefix = "summarize: "
def clean_text(text):
  sentences = nltk.sent_tokenize(text.strip())
  sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
  sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
                                 if len(sent) > 0 and
                                 sent[-1] in string.punctuation]
  text_cleaned = "\n".join(sentences_cleaned_no_titles)
  return text_cleaned

def preprocess_data(examples):
  texts_cleaned = [clean_text(text) for text in examples["article"]]
  inputs = [prefix + text for text in texts_cleaned]
  model_inputs = tokenizer(inputs, max_length=512, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["highlights"], max_length=64, 
                       truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [43]:
dataset_cleaned = CNN_dataset.filter(lambda example: (len(example['article']) >= 500) and (len(example['highlights']) >= 20))
tokenized_datasets = dataset_cleaned.map(preprocess_data, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

**Fine tuning T5**

In [44]:
batch_size = 8
model_name = "t5-small-essay-title-generation"
model_dir = f"Models/{model_name}"
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)
data_collator = DataCollatorForSeq2Seq(tokenizer)

PyTorch: setting up devices


In [45]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [46]:
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penal

In [47]:
trainer.train()
trainer.save_model()

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penal

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,2.187300,1.914121,27.224900,13.068700,22.739000,25.505500,18.999000
200,2.135500,1.879725,27.390400,13.178500,22.981400,25.681800,18.999000
300,2.071400,1.866549,27.335100,13.220500,23.018900,25.700900,19.000000
400,2.041800,1.855748,27.135000,13.166100,22.905700,25.508800,19.000000
500,2.038300,1.848319,27.287100,13.285800,23.051900,25.683500,18.999000
600,2.060100,1.839589,27.433700,13.359300,23.236200,25.756000,18.999000
700,2.028000,1.834677,27.423600,13.318600,23.165500,25.763100,18.999000
800,2.002100,1.832558,27.220900,13.171400,22.978600,25.569800,18.999000
900,1.999500,1.829454,27.291900,13.281900,23.115300,25.702300,18.999000
1000,1.991800,1.826929,27.384100,13.355000,23.133700,25.756200,18.999000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: article, highlights, id. If article, highlights, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: article, highlights, id. If article, highlights, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to Models/t5-small-essay-title-generation/checkpoint-200
Configuration saved in Models/t5-small-essay-title-generation/checkpoint-200/config.json
Model weights saved in Models/t5-small-essay-title-generation/checkpoint-200/pytorch_model.bin
tokenizer confi

In [48]:
model_name = "t5-small-essay-title-generation"
model_dir = f"Models/{model_name}"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
max_input_length = 512

Didn't find file Models/t5-small-essay-title-generation/spiece.model. We won't load it.
Didn't find file Models/t5-small-essay-title-generation/added_tokens.json. We won't load it.
loading file None
loading file Models/t5-small-essay-title-generation/tokenizer.json
loading file None
loading file Models/t5-small-essay-title-generation/special_tokens_map.json
loading file Models/t5-small-essay-title-generation/tokenizer_config.json
loading configuration file Models/t5-small-essay-title-generation/config.json
Model config T5Config {
  "_name_or_path": "Models/t5-small-essay-title-generation",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "nu

In [50]:
text = """
Expressions of intelligence in animal and human behaviour are so bountiful and so varied that there is an ontology
of associated abilities to name and study them, e.g. social intelligence, language, perception, knowledge representation,
planning, imagination, memory, and motor control. What could drive agents (natural or artificial) to behave intelligently in
such a diverse variety of ways?
One possible answer is that each ability arises from the pursuit of a goal that is designed specifically to elicit that ability.
For example, the ability of social intelligence has often been framed as the Nash equilibrium of a multi-agent system; the
ability of language by a combination of goals such as parsing, part-of-speech tagging, lexical analysis, and sentiment analysis;
and the ability of perception by object segmentation and recognition. In this paper, we consider an alternative hypothesis:
that the generic objective of maximising reward is enough to drive behaviour that exhibits most if not all abilities that are
studied in natural and artificial intelligence.

"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Session State and Callbacks in Streamlit

Expressions of intelligence in animal and human behaviour are so bountiful and so varied that there is an ontology of associated abilities to name and study them.


In [52]:
text = """
Today’s Internet users have fully embraced the Internet for
socializing and interacting with each other. It has been reported
that 92% of users go online daily [31]. Particularly, according
to recent findings from the Pew Research Center [16], 95% of
adolescents surveyed (ages 12-17) spend time online, reflecting
a high degree of user engagement, and 74% of them are
“mobile Internet users” who access the Internet on cell phones,
tablets, and other mobile devices at least occasionally.
The rise of social networks in the digital domain has led
to new definitions of friendships, relationships, and social
communications. However, one of the biggest issues of social
networks is their inherent potential to engender cyberbullying,
lmao.. & yurr reall funny
sk inny ass bitchh &.. hm..
that isn't really much of an
insult now is it? what if i
was fat? lol u suck at talkn
shit :] later white tr ash
skank :] ur super ugly nd
that guy u like really isnt
gonna come back around
for u.
(a) (b)
Fig. 1: Cyberbullying in text v.s. cyberbullying in an image. (a) shows a tweet with demeaning words and phrases.
(b) shows an image of a person showing a ‘loser’ hand gesture.
which has been widely recognized as a serious social problem. Multiple studies have suggested that cyberbullying can
have severe negative impact on an individual’s health, which
include deep emotional trauma, psychological and psychosomatic disorders [22], [78]. According to a National Crime
Prevention Council report, more than 40% of teenagers in
U.S. have reported being cyberbullied [60]. Dooley et al.
define cyberbullying as “Bullying via the Internet or mobile
phone” [39]. Cyberbullying encompasses all acts that are
aggressive, intentionally conducted by either a group or an
individual in cyberspace using information and communication
technologies (e.g. e-mail, text messages, chat rooms and social
networks) repeatedly or over time against victims who cannot
easily defend themselves [41].

"""

inputs = ["translate to german: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Session State and Callbacks in Streamlit

Die heutigen Internetnutzer haben das Internet voll und ganz in Anspruch genommen, um sich zu socialisieren und miteinander zu interagieren.


In [53]:
text = """
Today’s Internet users have fully embraced the Internet for
socializing and interacting with each other. It has been reported
that 92% of users go online daily [31]. Particularly, according
to recent findings from the Pew Research Center [16], 95% of
adolescents surveyed (ages 12-17) spend time online, reflecting
a high degree of user engagement, and 74% of them are
“mobile Internet users” who access the Internet on cell phones,
tablets, and other mobile devices at least occasionally.
The rise of social networks in the digital domain has led
to new definitions of friendships, relationships, and social
communications. However, one of the biggest issues of social
networks is their inherent potential to engender cyberbullying,
lmao.. & yurr reall funny
sk inny ass bitchh &.. hm..
that isn't really much of an
insult now is it? what if i
was fat? lol u suck at talkn
shit :] later white tr ash
skank :] ur super ugly nd
that guy u like really isnt
gonna come back around
for u.
(a) (b)
Fig. 1: Cyberbullying in text v.s. cyberbullying in an image. (a) shows a tweet with demeaning words and phrases.
(b) shows an image of a person showing a ‘loser’ hand gesture.
which has been widely recognized as a serious social problem. Multiple studies have suggested that cyberbullying can
have severe negative impact on an individual’s health, which
include deep emotional trauma, psychological and psychosomatic disorders [22], [78]. According to a National Crime
Prevention Council report, more than 40% of teenagers in
U.S. have reported being cyberbullied [60]. Dooley et al.
define cyberbullying as “Bullying via the Internet or mobile
phone” [39]. Cyberbullying encompasses all acts that are
aggressive, intentionally conducted by either a group or an
individual in cyberspace using information and communication
technologies (e.g. e-mail, text messages, chat rooms and social
networks) repeatedly or over time against victims who cannot
easily defend themselves [41].

"""

inputs = ["translate to french: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Session State and Callbacks in Streamlit

Les internautes d’aujourd’hui ont pleinement adopté l’Internet pour socialiser et interagir entre eux.


In [56]:
text = """ 
An age-old vendetta between two powerful families erupts into bloodshed. A group of masked Montagues risk further conflict by gatecrashing a Capulet party. A young lovesick Romeo Montague falls instantly in love with Juliet Capulet, who is due to marry her father’s choice, the County Paris. With the help of Juliet’s nurse, the women arrange for the couple to marry the next day, but Romeo’s attempt to halt a street fight leads to the death of Juliet’s own cousin, Tybalt, for which Romeo is banished. In a desperate attempt to be reunited with Romeo, Juliet follows the Friar’s plot and fakes her own death. The message fails to reach Romeo, and believing Juliet dead, he takes his life in her tomb. Juliet wakes to find Romeo’s corpse beside her and kills herself. The grieving family agree to end their feud.

"""
inputs = ["translate to german: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Session State and Callbacks in Streamlit

Eine jahrhundertealte vendetta zwischen zwei mächtigen Familien erupts in Blutvergießen.


In [57]:
text = """
The rise of social media has significantly influenced our 
lives. However, this puts adolescents at risk to becoming 
victims of online misconduct, especially cyberbullying. 
Cyberbullying refers to an aggressive, intentional act 
conducted by either a group or an individual in cyberspace 
using information and communication technologies (e.g. email, mobile phone, and social networks) repeatedly or over 
time against victims who cannot easily defend themselves [1]. 
According to a 2015 report from the Cyberbullying Research 
Center, about one-third of the high school students from 
random samples have experienced cyberbullying [2]. 
Unlike in traditional bullying, techniques and forms used 
by cyberbullies change rapidly and is more harmful and harder 
to detect [3]. For example, it is easy to anonymously spread 
rumors about people online, and there is a low risk of being 
caught. Thus, it is necessary to detect cyberbullying in order to 
protect adolescents. 
Unlike video and image based methods, text based 
cyberbullying is the most common form used by perpetrators. 
Moreover, other forms are usually combined with bullying text. 
Thus, we focus on detecting textual cyberbullying in this study. 
Textual cyberbullying detection methods can be divided into 
two categories: keywords based and artificial intelligence (AI) 
based [4].

"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# A paragraph from a research paper on Cyberbullying

Cyberbullying refers to an aggressive, intentional act conducted by either a group or an individual in cyberspace using information and communication technologies (e.g.


In [58]:
text = """
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 Englishto-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.8 after
training for 3.5 days on eight GPUs, a small fraction of the training costs of the
best models from the literature. We show that the Transformer generalizes well to
other tasks by applying it successfully to English constituency parsing both with
large and limited training data

"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# A paragraph from research paper "Attention is all you need"

We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.


In [59]:
text = """
When abilities associated with intelligence arise as solutions to a singular goal of reward maximisation, this may in fact
provide a deeper understanding since it explains why such an ability arises (e.g. that classification of crocodiles is important
to avoid being eaten). In contrast, when each ability is understood as the solution to its own specialised goal, the why question is side-stepped in order to focus upon what that ability does (e.g. discriminating crocodiles from logs). Furthermore, a
singular goal may also provide a broader understanding of each ability that may include characteristics that are otherwise
hard to formalise, such as dealing with irrational agents in social intelligence (e.g. pacifying an angry aggressor), grounding
language to perceptual experience (e.g. dialogue regarding the best way to peel a fruit), or understanding haptics in perception (e.g. picking a sharp object from a pocket). Finally, implementing abilities in service of a singular goal, rather than for
their own specialised goals, also answers the question of how to integrate abilities, which otherwise remains an outstanding
issue

"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# A Paragraph from research paper "Reward is Enough"


a singular goal may also provide a broader understanding of each ability that may include characteristics that are otherwise hard to formalise.


In [63]:
text = """
On the continent of Roshar, on a planet of the same name, events called Desolations periodically reduce humans to a Neolithic stage of development and population. In the Roshar religion, Vorinism, the Almighty sent 10 Heralds to help humanity. These divine messengers formed the Knights Radiant, warriors with magical abilities, to protect humans from Voidbringers, the monsters who cause Desolations. According to Vorin teachings, the Heralds and Knights betrayed their oaths and deserted mankind, exhausted by cyclical war and suffering.



Thousands of years after the last Desolation, the people on Roshar are reluctant to believe that a new cataclysmic event might soon take place. Several characters from different countries and social strata become aware of the looming danger and develop the ability to channel stormlight, a kind of radiant energy, which helps them perform magic. The narrative rotates between their points of view as they are slowly drawn together at the Shattered Plains, where the most powerful nation on Roshar, Alethkar, is engaged in a long war with the mysterious Parshendi people. The main characters include Kaladin, a surgeon’s son who becomes a soldier and later a slave; Shallan Davar, a young aristocratic woman set on saving her family from financial ruin; Dalinar Kholin, a fierce warrior and the current Alethi king’s uncle; and Szeth-son-son-Vallano, an outcast of the Shin nation cursed to obey whomever possesses his Oathstone.



An unknown Parshendi orders Szeth to assassinate Gavilar, the King of Alethkar. Devastated by his brother’s death, Dalinar vows to destroy the Parshendi. After six years of pointless fighting, he begins to doubt the war effort and the ways of Alethi society as a whole. His change of heart is inspired by an ancient text, The Way of Kings, which Gavilar had become interested in before his death. The book refutes the aggressive and competitive Alethi culture. Additionally, Dalinar experiences strange visions that warn him of coming danger and compel him to unite his people. Dalinar’s visions also challenge traditional Vorin teachings, placing him in a politically sensitive situation and making his attempts to unite the Alethi Highlords seem hopeless.



Meanwhile, Kaladin, a gifted young warrior, is betrayed by his commander and made a slave. Kaladin becomes a bridgeman on the Shattered Plains, whose job is to help the army cross over the chasms separating individual plateaus on the Shattered Palins by drawing enemy fire. A spirit, called a spren, inspires Kaladin to be a better person and to help those around him. Kaladin becomes the crew leader and finds ingenious ways to help his men stay alive. At the same time, he begins to develop the ability to absorb stormlight, which gives him special powers such as walking up walls. At the end of the novel, the bridge crew saves Dalinar and his remaining troops from a trap set by another Highprince. As a reward, Dalinar buys all the bridgemen and liberates them.



Shallan seeks to gain the confidence of Jasnah Kholin, Gavilar’s daughter. Jasnah is a renowned scholar, who also possesses a Soulcaster, a powerful magical artifact that allows her to transform objects. Shallan plans to steal the Soulcaster in order to help her family prevent bankruptcy. The young woman succeeds in substituting the Soulcaster with a broken one, but after being poisoned by an assassin she reveals her deception so that Jasnah can save her life with magic. Afterward, Shallan realizes that Jasnah does not actually need an artifact to perform magic and that Shallan herself has a similar ability. The two women reconcile, and Jasnah confides in her protégé that the Voidbringers are most likely the Parshendi. The two women decide to travel to the Shattered Plains in order to seek answers and prevent the Desolation."""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# A Paragraph from Book 1 of THE STROMLIGHT ARCHIVE "Way of Kings"


In the Roshar religion, Vorinism, the Almighty sent 10 Heralds to help humanity.


** References **
1. Towdards Data science(The FLow of the code is dervied from this source but not the dataset fitting and tuning)
